---
title: "Vortex–Sentiment Adaptive Volatility (VSAV) Strategy"
author:
  - name: Group Data Oracles
    affiliations:
      - name: Boston University
        city: Boston
        state: MA
format:
  html:
    toc: true
    css: styles.css
    html-math-method: katex
    embed-resources: false
    code-fold: true
jupyter: python3
execute:
  eval: true
---


## Importing Necessary Libraries for Analysis

In [1]:
import yfinance as yf  # For downloading financial data
import numpy as np      # For numerical operations
import pandas as pd     # For data manipulation
import requests # For downloading the API data
import numpy as np 
import plotly.graph_objects as go
import plotly.express as px # Import the Plotly Express module for interactive visualization
import json
import vectorbt as vbt
from plotly.subplots import make_subplots
import streamlit as st

import plotly.io as pio
pio.renderers.default = 'iframe_connected'

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Data Collection

### Fetch daily OHLCV data 

In [2]:
# Data for the TSLA, XLY, and SPY tickers is retrieved from the Yahoo Finance library, covering the period from January 1, 2019, 
# to March 5, 2025.
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05') 
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
def multiindex_to_singleindex(df):
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    return df

In [4]:
tsla = multiindex_to_singleindex(tsla)
spy = multiindex_to_singleindex(spy)
xly = multiindex_to_singleindex(xly)

In [5]:
# Displays a summary of the TSLA DataFrame, including column names, data types, non-null counts, and memory usage.
tsla.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Close_TSLA   1551 non-null   float64
 1   High_TSLA    1551 non-null   float64
 2   Low_TSLA     1551 non-null   float64
 3   Open_TSLA    1551 non-null   float64
 4   Volume_TSLA  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [6]:
# Displays a summary of the XLY DataFrame, including column names, data types, non-null counts, and memory usage.
xly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Close_XLY   1551 non-null   float64
 1   High_XLY    1551 non-null   float64
 2   Low_XLY     1551 non-null   float64
 3   Open_XLY    1551 non-null   float64
 4   Volume_XLY  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [7]:
# Displays a summary of the SPY DataFrame, including column names, data types, non-null counts, and memory usage.
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Close_SPY   1551 non-null   float64
 1   High_SPY    1551 non-null   float64
 2   Low_SPY     1551 non-null   float64
 3   Open_SPY    1551 non-null   float64
 4   Volume_SPY  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


### Fetch sentiment scores from the API

In [8]:
def get_news_sentiment(ticker, start_date, end_date, limit, api_key):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&time_from={start_date}&time_to={end_date}&limit={limit}&tickers={ticker}&apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        sentiment_data = response.json()

        with open(f'{ticker}_sentiment_raw.json', "w") as f:
            json.dump(sentiment_data, f, indent=4)
        # sentiment_df = pd.DataFrame(sentiment_data['feed'])
        return "Full sentiment JSON saved ✅"
    else:
        print("API call failed:", response.status_code)
        return None

In [9]:
get_news_sentiment('TSLA', '20250101T0130', '20250301T0130', 1000, 'PNM5EHRALIOT1CKJ')

'Full sentiment JSON saved ✅'

## Indicator Calculation

### Compute VI+ and VI-

In [10]:
# Defines a function to calculate the Vortex Indicator (VI) for a given DataFrame and ticker symbol.
# The calculation uses a default lookback period of 14 days unless specified otherwise.
def calculate_vortex(df, value, n=14):
    # Extracts the high, low, and close price series for the specified ticker.
    high = df[("High_"+value)]
    low = df[("Low_"+value)]
    close = df[("Close_"+value)]

    # Calculates the Vortex Movement values:
    # VM+ = absolute difference between today's high and yesterday's low
    # VM− = absolute difference between today's low and yesterday's high
    vm_plus = abs(high - low.shift(1))     # |Today's High – Yesterday’s Low|
    vm_minus = abs(low - high.shift(1))    # |Today's Low – Yesterday’s High|

    # Computes the True Range (TR) as the maximum of:
    # - High - Low
    # - Absolute difference between High and Previous Close
    # - Absolute difference between Low and Previous Close
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Applies a rolling window to compute the n-period sum of VM+ and VM− values
    # and the corresponding True Range values.
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Calculates the Vortex Indicator components:
    # VI+ = sum of VM+ over n periods divided by sum of TR over n periods
    # VI− = sum of VM− over n periods divided by sum of TR over n periods
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    # Returns the VI+ and VI− series as output.
    return vi_plus, vi_minus

In [11]:
# Calculates the Vortex Indicator values for TSLA and stores the results as new columns in the DataFrame.
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')

# Calculates the Vortex Indicator values for XLY and stores the results as new columns in the DataFrame.
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')

# Calculates the Vortex Indicator values for SPY and stores the results as new columns in the DataFrame.
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [12]:
# Displays the first 20 rows of the TSLA DataFrame to provide an initial overview of its structure and content with the new function applied.
tsla.head(20)

,Close_TSLA,High_TSLA,Low_TSLA,Open_TSLA,Volume_TSLA,VI+,VI-
Date,,,,,,,
2019-01-02,20.674667,21.008667,19.920000,20.406668,174879000,NaN,NaN
2019-01-03,20.024000,20.626667,19.825333,20.466667,104478000,NaN,NaN
2019-01-04,21.179333,21.200001,20.181999,20.400000,110911500,NaN,NaN
2019-01-07,22.330667,22.449333,21.183332,21.448000,113268000,NaN,NaN
2019-01-08,22.356667,22.934000,21.801332,22.797333,105127500,NaN,NaN
2019-01-09,22.568666,22.900000,22.098000,22.366667,81493500,NaN,NaN
2019-01-10,22.997999,23.025999,22.119333,22.293333,90846000,NaN,NaN
2019-01-11,23.150667,23.227333,22.584667,22.806000,75586500,NaN,NaN
2019-01-14,22.293333,22.833332,22.266666,22.825333,78709500,NaN,NaN


### Calculate Volume-Weighted Sentiment 

In [13]:
def json_reader(ticker):
    with open(f'{ticker}_sentiment_raw.json', "r") as f:
        sentiment_json_ticker = json.load(f)
        sentiment_feed = sentiment_json_ticker.get("feed", [])
        sentiment_data = []
        # Iterate through each item in the sentiment feed to extract relevant fields
        for item in sentiment_feed:
            try:
                sentiment_data.append({
                    # Convert the timestamp to pandas datetime for proper indexing
                    "time_published": pd.to_datetime(item["time_published"]),
                    # Convert the sentiment score string to float
                    "sentiment_score": float(item["overall_sentiment_score"]),
                    # Store the sentiment label (e.g., Positive, Neutral, Negative)
                    "sentiment_label": item["overall_sentiment_label"],
                })
            except (KeyError, ValueError, TypeError):
                # Skip malformed or incomplete entries that raise an error
                continue    
        # Convert the structured list of dictionaries into a pandas DataFrame
        sentiment_df = pd.DataFrame(sentiment_data)
        # Set the 'time_published' column as the DataFrame index to enable time-series operations
        # sentiment_df.set_index("time_published", inplace=True)
        sentiment_df['time_published']= pd.to_datetime(sentiment_df['time_published'].dt.date)
    return sentiment_df
    # globals()[f"{ticker.lower()}_sentiment_data"] = sentiment_data

In [14]:
tsla_sentiment_df = json_reader('TSLA')

In [15]:
tsla_sentiment_df.head()

,time_published,sentiment_score,sentiment_label
0,2025-03-01,0.225994,Somewhat-Bullish
1,2025-02-28,-0.098739,Neutral
2,2025-02-28,-0.041235,Neutral
3,2025-02-28,-0.038786,Neutral
4,2025-02-28,0.021961,Neutral


In [16]:
tsla_sentiment_scores_filtered = tsla_sentiment_df[(tsla_sentiment_df['time_published']).isin(tsla.index)]
tsla_sentiment_scores_filtered = tsla_sentiment_scores_filtered.groupby('time_published')['sentiment_score'].mean().reset_index()

In [17]:
tsla_merged_data = pd.merge(
    tsla['Volume_TSLA'].reset_index().rename(columns={'Volume_TSLA': 'Volume'}),
    tsla_sentiment_scores_filtered,
    left_on='Date',
    right_on='time_published',
    how='inner'
)
# Compute the weighted sentiment by multiplying raw sentiment by trading volume
tsla_merged_data['Weighted_Sentiment'] = tsla_merged_data['Volume'] * tsla_merged_data['sentiment_score']

# Calculate a 5-day rolling average of the weighted sentiment to smooth short-term noise
tsla_merged_data['5_day_avg_sentiment'] = tsla_merged_data['Weighted_Sentiment'].rolling(window=5).mean()

# Define a binary condition for when the average sentiment is positive
tsla_merged_data['Buy_Condition'] = tsla_merged_data['5_day_avg_sentiment'] > 0

# Normalize the rolling sentiment score by average volume to allow comparability across scales
tsla_merged_data['5_day_avg_sentiment_norm'] = (
    tsla_merged_data['5_day_avg_sentiment'] / tsla_merged_data['Volume'].mean()
)

In [18]:
tsla_merged_data.head()

,Date,Volume,time_published,sentiment_score,Weighted_Sentiment,5_day_avg_sentiment,Buy_Condition,5_day_avg_sentiment_norm
0,2025-01-31,83568200,2025-01-31,0.194614,1.626354e+07,NaN,False,NaN
1,2025-02-03,93732100,2025-02-03,0.129243,1.211426e+07,NaN,False,NaN
2,2025-02-04,57072200,2025-02-04,0.173107,9.879602e+06,NaN,False,NaN
3,2025-02-05,57223300,2025-02-05,0.136874,7.832396e+06,NaN,False,NaN
4,2025-02-06,77918200,2025-02-06,0.118095,9.201782e+06,1.105832e+07,True,0.132787


### Derive ATR (10) for Volatility Adjustments

In [19]:
def calculate_true_range(df, ticker):
    df["prev_close"] = df[f'Close_{ticker}'].shift(1)
    df["tr1"] = df[f'High_{ticker}'] - df[f'Low_{ticker}']
    df["tr2"] = abs(df[f'High_{ticker}'] - df["prev_close"])
    df["tr3"] = abs(df[f'Low_{ticker}'] - df["prev_close"])
    df["true_range"] = df[["tr1", "tr2", "tr3"]].max(axis=1)
    df["ATR_10"] = df["true_range"].rolling(window=10).mean()
    df["atr_pct"] = df["ATR_10"] / df[f'Close_{ticker}']
    return df

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

In [20]:
tsla = calculate_true_range(tsla, 'TSLA')
tsla["position_size"] = tsla.apply(position_size, axis=1)

# ---- Preview ----
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))

            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [21]:
# Create a line chart to visualize the ATR% (Average True Range as a percentage of price) over time
fig_atr_tsla = px.line(tsla, x=tsla.index, y="atr_pct", title="ATR% Over Time")

# Add a horizontal reference line at 3% to represent the low-volatility cutoff threshold
fig_atr_tsla.add_hline(
    y=0.03, 
    line_dash="dot", 
    line_color="green", 
    annotation_text="Low Volatility Cutoff"
)

# Display the chart
fig_atr_tsla.show()
# Display in Streamlit
# st.subheader("ATR% Over Time for TSLA")
# st.plotly_chart(fig_atr_tsla, use_container_width=True)

The chart illustrates the historical volatility of TSLA, measured by the Average True Range (ATR) as a percentage of the closing price. Periods where the ATR% falls below the dotted green line at 3% indicate low volatility, which is typically associated with more stable market conditions. In contrast, noticeable spikes—such as those seen in 2020 and 2021—reflect periods of heightened volatility. More recently, ATR% values appear to remain closer to or slightly above the low-volatility threshold, suggesting relatively calmer market behavior compared to earlier years.

In [22]:
# Filter the TSLA DataFrame to include only records from the year 2025
tsla_2025 = tsla[tsla.index.year == 2025]

# Create a line chart to visualize ATR% for TSLA during 2025
fig = px.line(
    tsla_2025,
    x=tsla_2025.index,
    y="atr_pct",
    title="TSLA ATR% Over Time (2025 Only)"
)

# Add a horizontal line at the 3% threshold to denote the low-volatility cutoff
fig.add_hline(
    y=0.03,
    line_dash="dot",
    line_color="green",
    annotation_text="Low Volatility Cutoff"
)

# Display the chart
fig.show()

The chart displays ATR% for TSLA during 2025, reflecting how the stock's volatility has evolved since the start of the year. While ATR% began above the 7% mark in early January, it gradually declined and remained mostly between 4% and 6% throughout February. Although volatility did not breach the low-volatility threshold of 3%, the dip toward that level suggests a period of relative calm. Toward early March, ATR% showed a clear upward trend, indicating a potential resurgence in market volatility.

In [23]:
def signal_generation(df, ticker):
    df['atr_pct'] = df['ATR_10'] / df['Close_' + ticker]

    # Create Buy Signal (assuming VI_Cross_Up is defined elsewhere)
    df['Buy_Signal'] = df['VI+'] > df['VI-']  # Vortex crossover
    # + add any other buy conditions here...

    # Create Sell Signal (basic)
    df['Sell_Signal'] = df['VI-'] > df['VI+']

    # Initialize position state
    df['Position'] = 0
    peak_price = 0

    for i in range(1, len(df)):
        if df['Buy_Signal'].iloc[i]:
            df.at[df.index[i], 'Position'] = 1
            peak_price = df['Close_' + ticker].iloc[i]
        elif df['Position'].iloc[i - 1] == 1:
            current_price = df['Close_' + ticker].iloc[i]
            peak_price = max(peak_price, current_price)
            drawdown = (peak_price - current_price) / peak_price

            if drawdown >= 0.03:
                df.at[df.index[i], 'Sell_Signal'] = True  # trailing stop
                df.at[df.index[i], 'Position'] = 0
            else:
                df.at[df.index[i], 'Position'] = 1    
    return df

In [24]:
tsla = signal_generation(tsla, 'TSLA')
# Display the total number of buy and sell signals generated across the dataset
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())

Buy signals: 857
Sell signals: 680


In [25]:
# Create an empty figure object
fig = go.Figure()

# Plot the TSLA closing price as a continuous line
fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla['Close_TSLA'],
    mode='lines',
    name='TSLA Price'
))

# Add markers to indicate Buy Signals using upward-pointing green triangles
fig.add_trace(go.Scatter(
    x=tsla[tsla['Buy_Signal']].index,
    y=tsla[tsla['Buy_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-up', size=10, color='green'),
    name='Buy Signal'
))

# Add markers to indicate Sell Signals using downward-pointing red triangles
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    marker=dict(symbol='triangle-down', size=10, color='red'),
    name='Sell Signal'
))

# Update layout settings including title and visual style
fig.update_layout(
    title='TSLA Buy & Sell Signals',
    template='plotly_white'
)

# Render the interactive plot
fig.show()

The chart illustrates the closing price of Tesla stock over time, with overlaid trading signals generated by the strategy. Green upward triangles represent buy signals, while red downward triangles mark sell signals. These signals are distributed throughout periods of both rising and falling prices, reflecting how the algorithm dynamically enters and exits positions based on market conditions. Clusters of signals during high-volatility periods—such as 2020, 2021, and early 2025—indicate frequent entries and exits, whereas more stable phases show fewer trades.

In [26]:
# Calculate ATR as a percentage of the closing price to normalize volatility
tsla['atr_pct'] = tsla['ATR_10'] / tsla['Close_TSLA']

# Define Vortex Indicator crossover signals:
# - VI_Cross_Up: Identifies when VI+ crosses above VI− (potential bullish signal)
# - VI_Cross_Down: Identifies when VI− crosses above VI+ (potential bearish signal)
tsla['VI_Cross_Up'] = (tsla['VI+'] > tsla['VI-']) & (tsla['VI+'].shift(1) <= tsla['VI-'].shift(1))
tsla['VI_Cross_Down'] = (tsla['VI-'] > tsla['VI+']) & (tsla['VI-'].shift(1) <= tsla['VI+'].shift(1))

# Initialize signal and state columns
tsla['Buy_Signal'] = False          # Flag for buy signal
tsla['Sell_Signal'] = False         # Flag for sell signal
tsla['Position'] = 0                # Position state: 1 = in position, 0 = no position
tsla['Entry_Type'] = None           # Strategy classification: 'aggressive' or 'conservative'

# Initialize control variables for trailing stop and price tracking
in_position = False                 # Boolean flag for current position state
peak_price = 0                      # Highest price observed during an open position

# Iterate through the DataFrame to simulate trading logic based on Vortex signals and volatility
for i in range(1, len(tsla)):
    row = tsla.iloc[i]
    idx = tsla.index[i]

    # Buy condition: Enter a new position if VI_Cross_Up occurs and no current position is held
    if not in_position and row['VI_Cross_Up']:
        tsla.at[idx, 'Buy_Signal'] = True
        tsla.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_TSLA']

        # Classify entry type based on volatility threshold
        if row['atr_pct'] < 0.03:
            tsla.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            tsla.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, evaluate for trailing stop or VI_Cross_Down exit condition
    elif in_position:
        current_price = row['Close_TSLA']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        # Sell condition: Exit if drawdown exceeds 3% or VI_Cross_Down occurs
        if drawdown >= 0.03 or row['VI_Cross_Down']:
            tsla.at[idx, 'Sell_Signal'] = True
            tsla.at[idx, 'Position'] = 0
            in_position = False
        else:
            tsla.at[idx, 'Position'] = 1  # Maintain position

# Output the total count of each type of signal and entry classification
print("Buy signals:", tsla['Buy_Signal'].sum())
print("Sell signals:", tsla['Sell_Signal'].sum())
print("Aggressive entries:", (tsla['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (tsla['Entry_Type'] == 'conservative').sum())

Buy signals: 80
Sell signals: 80
Aggressive entries: 5
Conservative entries: 75


In [27]:
# Create an empty figure to hold all plot layers
fig = go.Figure()

# Plot the tsla closing price as a continuous blue line
fig.add_trace(go.Scatter(
    x=tsla.index,
    y=tsla['Close_TSLA'],
    mode='lines',
    name='TSLA Price',
    line=dict(color='blue')
))

# Add markers for aggressive buy signals (Entry_Type = 'aggressive')
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'aggressive')]['Close_TSLA'],
    mode='markers',
    name='Buy (Aggressive)',
    marker=dict(symbol='triangle-up', color='limegreen', size=10)
))

# Add markers for conservative buy signals (Entry_Type = 'conservative')
fig.add_trace(go.Scatter(
    x=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')].index,
    y=tsla[(tsla['Buy_Signal']) & (tsla['Entry_Type'] == 'conservative')]['Close_TSLA'],
    mode='markers',
    name='Buy (Conservative)',
    marker=dict(symbol='triangle-up', color='green', size=10)
))

# Add markers for sell signals using red downward-pointing triangles
fig.add_trace(go.Scatter(
    x=tsla[tsla['Sell_Signal']].index,
    y=tsla[tsla['Sell_Signal']]['Close_TSLA'],
    mode='markers',
    name='Sell Signal',
    marker=dict(symbol='triangle-down', color='red', size=10)
))

# Configure chart layout with appropriate title, axis labels, and style
fig.update_layout(
    title='TSLA Buy/Sell Signals Over Time',
    xaxis_title='Date',
    yaxis_title='Price (USD)',
    template='plotly_white',
    height=600
)

# Render the figure
fig.show()

The chart displays the historical closing price of Tesla (TSLA) stock alongside algorithmically generated buy and sell signals. The blue line represents TSLA's closing price, while the green upward-pointing triangles indicate buy entries—distinguished by lime green for aggressive entries (lower volatility) and dark green for conservative entries (higher volatility). Red downward-pointing triangles represent sell signals.

The buy signals are generally aligned with upward momentum, and sell signals frequently follow periods of short-term retracement or heightened volatility. The system shows particularly dense activity around highly volatile phases, such as mid-2020 to early 2022, capturing many entries and exits. In contrast, during more stable periods, the signals are more spaced out. Overall, the plot provides a clear visual assessment of how the strategy adapts dynamically to changing market conditions by modulating its entries based on volatility and exiting with protective trailing logic.

## Tesla Analysis Results

In [28]:
tsla_merged_data = pd.merge(
    tsla_merged_data, 
    tsla[['Close_TSLA', 'High_TSLA', 'Low_TSLA', 'Open_TSLA', 'Volume_TSLA',
          'VI+', 'VI-', 'prev_close', 'tr1', 'tr2', 'tr3', 'true_range', 'ATR_10', 'position_size']], 
    on='Date', 
    how='left')
tsla_merged_data.head()

,Date,Volume,time_published,sentiment_score,Weighted_Sentiment,5_day_avg_sentiment,Buy_Condition,5_day_avg_sentiment_norm,Close_TSLA,High_TSLA,...,Volume_TSLA,VI+,VI-,prev_close,tr1,tr2,tr3,true_range,ATR_10,position_size
0,2025-01-31,83568200,2025-01-31,0.194614,1.626354e+07,NaN,False,NaN,404.600006,419.989990,...,83568200,1.012243,0.857439,400.279999,18.649994,19.709991,1.059998,19.709991,18.478998,0.005
1,2025-02-03,93732100,2025-02-03,0.129243,1.211426e+07,NaN,False,NaN,383.679993,389.170013,...,93732100,0.941453,0.927890,404.600006,14.810028,15.429993,30.240021,30.240021,18.911002,0.005
2,2025-02-04,57072200,2025-02-04,0.173107,9.879602e+06,NaN,False,NaN,392.209991,394.000000,...,57072200,0.911693,0.973944,383.679993,12.600006,10.320007,2.279999,12.600006,17.482001,0.005
3,2025-02-05,57223300,2025-02-05,0.136874,7.832396e+06,NaN,False,NaN,378.170013,388.390015,...,57223300,0.862377,1.041572,392.209991,12.860016,3.819977,16.679993,16.679993,17.809000,0.005
4,2025-02-06,77918200,2025-02-06,0.118095,9.201782e+06,1.105832e+07,True,0.132787,374.320007,375.399994,...,77918200,0.805785,1.075550,378.170013,12.220001,2.770020,14.990021,14.990021,18.130002,0.005


In [29]:
# Calculate ATR percentage
tsla_merged_data['atr_pct'] = tsla_merged_data['ATR_10'] / tsla_merged_data['Close_TSLA']

# Vortex crossover logic
tsla_merged_data['VI_Cross_Up'] = (tsla_merged_data['VI+'] > tsla_merged_data['VI-']) & (tsla_merged_data['VI+'].shift(1) <= tsla_merged_data['VI-'].shift(1))
tsla_merged_data['VI_Cross_Down'] = (tsla_merged_data['VI-'] > tsla_merged_data['VI+']) & (tsla_merged_data['VI-'].shift(1) <= tsla_merged_data['VI+'].shift(1))

# Initialize signal & state columns
tsla_merged_data['Buy_Signal'] = False
tsla_merged_data['Sell_Signal'] = False
tsla_merged_data['Position'] = 0
tsla_merged_data['Entry_Type'] = None  # aggressive/conservative

# Trailing stop logic variables
in_position = False
peak_price = 0

for i in range(1, len(tsla_merged_data)):
    row = tsla_merged_data.iloc[i]
    idx = tsla_merged_data.index[i]
    # Buy condition
    if not in_position or row['VI_Cross_Up'] or row['5_day_avg_sentiment_norm']>0:
        tsla_merged_data.at[idx, 'Buy_Signal'] = True
        tsla_merged_data.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_TSLA']

        # Entry Type: aggressive if ATR < 3%, else conservative
        if row['atr_pct'] < 0.03:
            tsla_merged_data.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            tsla_merged_data.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, check for trailing stop or VI cross down
    elif in_position:
        current_price = row['Close_TSLA']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03 or row['VI_Cross_Down']:
            tsla_merged_data.at[idx, 'Sell_Signal'] = True
            tsla_merged_data.at[idx, 'Position'] = 0
            in_position = False
        else:
            tsla_merged_data.at[idx, 'Position'] = 1

# Show result counts
print("Buy signals:", tsla_merged_data['Buy_Signal'].sum())
print("Sell signals:", tsla_merged_data['Sell_Signal'].sum())
print("Aggressive entries:", (tsla_merged_data['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (tsla_merged_data['Entry_Type'] == 'conservative').sum())


Buy signals: 18
Sell signals: 1
Aggressive entries: 0
Conservative entries: 18


In [30]:
# Ensure 'Date' is datetime and set as index if needed
tsla_merged_data['Date'] = pd.to_datetime(tsla_merged_data['Date'])

fig = go.Figure()

# Plot 5-day Avg Sentiment
fig.add_trace(go.Scatter(
    x=tsla_merged_data['Date'],
    y=tsla_merged_data['5_day_avg_sentiment_norm'],
    mode='lines+markers',
    name='5-Day Avg Sentiment',
    line=dict(color='blue')
))

# Plot ATR %
fig.add_trace(go.Scatter(
    x=tsla_merged_data['Date'],
    y=tsla_merged_data['atr_pct'],
    mode='lines+markers',
    name='ATR %',
    yaxis='y2',
    line=dict(color='orange')
))

# Optional: Highlight Buy Signal Dates (even though there are none now)
fig.add_trace(go.Scatter(
    x=tsla_merged_data.loc[tsla_merged_data['Buy_Signal'], 'Date'],
    y=tsla_merged_data.loc[tsla_merged_data['Buy_Signal'], '5_day_avg_sentiment_norm'],
    mode='markers',
    marker=dict(color='green', size=10, symbol='star'),
    name='Buy Signal'
))

# Add dual axis layout
fig.update_layout(
    title="5-Day Sentiment vs ATR % (with Buy Signals)",
    xaxis_title='Date',
    yaxis=dict(title='5-Day Avg Sentiment'),
    yaxis2=dict(title='ATR %', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.show()

In [31]:
def backtest(df, ticker):
    capital = 100000
    in_position = False
    entry_price = 0
    position_value = 0
    cash = capital
    returns = []

    for i in range(len(df)):
        row = df.iloc[i]

        if row['Buy_Signal'] and not in_position:
            position_size = row['position_size']
            position_value = cash * position_size
            entry_price = row['Close_' + ticker]
            shares_bought = position_value / entry_price
            cash -= position_value
            in_position = True
        elif row['Sell_Signal'] and in_position:
            exit_price = row['Close_' + ticker]
            proceeds = shares_bought * exit_price
            profit = proceeds - position_value
            cash += proceeds
            returns.append(profit)
            in_position = False
            position_value = 0
            entry_price = 0

    final_value = cash + (shares_bought * row['Close_' + ticker] if in_position else 0)
    total_return = final_value - capital
    result = f"Final Capital: ${final_value:.2f} \nTotal Return: ${total_return:.2f} \nTotal Trades: {len(returns)}\nAverage Profit per Trade: ${np.mean(returns):.2f}"
    return result

In [32]:
print(backtest(tsla_merged_data, 'TSLA')) #w/ sentiment data

Final Capital: $99898.47 
Total Return: $-101.53 
Total Trades: 1
Average Profit per Trade: $11.12


In [33]:
print(backtest(tsla, 'TSLA')) #w/o sentiment data

Final Capital: $100575.32 
Total Return: $575.32 
Total Trades: 80
Average Profit per Trade: $7.19


In [34]:
def f_portfolio(df, ticker):
    df = df.dropna(subset=['Close_' + ticker])
    entries = df['Buy_Signal'].astype(bool)
    exits = df['Sell_Signal'].astype(bool)

    price = df['Close_' + ticker]
    portfolio = vbt.Portfolio.from_signals(
        close=price,
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )
    return portfolio

In [35]:
# without centiment data
tsla_portfolio = f_portfolio(tsla, 'TSLA')

print(tsla_portfolio.stats())
tsla_portfolio.plot().show()

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           162759.235978
Total Return [%]                        62.759236
Benchmark Return [%]                  1215.813231
Max Gross Exposure [%]                      100.0
Total Fees Paid                      24054.581607
Max Drawdown [%]                        55.348959
Max Drawdown Duration                       730.0
Total Trades                                   80
Total Closed Trades                            80
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                                 32.5
Best Trade [%]                          46.283397
Worst Trade [%]                         -9.410141
Avg Winning Trade [%]                   11.344578
Avg Losing Trade [%]                    -3.847352


The backtest results show that while the strategy achieved a total return of approximately 62.76%, it significantly underperformed compared to a simple buy-and-hold strategy on TSLA, which yielded a 1215.81% return. The strategy executed 80 trades with a low win rate of 32.5%, indicating that most trades were unprofitable. Although it had a few strong winners, the average profit per trade was marginal, with a profit factor of 1.19. Additionally, the portfolio experienced a substantial maximum drawdown of 55.35% and a prolonged recovery period lasting two years, signaling high risk. Visuals further confirm that many trades resulted in small losses or gains, with only a few notable profitable exits. Overall, while the strategy demonstrates some profitability, its risk-return profile is weak and may require optimization in entry/exit logic, volatility filtering, or sentiment integration to compete with the benchmark performance.

## XLY Analysis Results

In [36]:
xly = calculate_true_range(xly, 'XLY')
xly["position_size"] = xly.apply(position_size, axis=1)

# ---- Preview ----
print(xly[["Close_XLY", "ATR_10", "atr_pct", "position_size"]].tail(10))

             Close_XLY    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  225.618988  2.870099  0.012721           0.01
2025-02-20  223.674316  2.919964  0.013055           0.01
2025-02-21  217.790527  3.453495  0.015857           0.01
2025-02-24  216.972778  3.270997  0.015076           0.01
2025-02-25  215.835892  3.511334  0.016269           0.01
2025-02-26  214.948349  3.602083  0.016758           0.01
2025-02-27  211.846878  3.751672  0.017709           0.01
2025-02-28  215.367203  3.836439  0.017813           0.01
2025-03-03  211.398117  4.429805  0.020955           0.01
2025-03-04  207.668396  4.845659  0.023334           0.01


In [37]:
fig = px.line(xly, x=xly.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()

In [38]:
# Filter only 2025 data
xly_2025 = xly[xly.index.year == 2025]

# Plot
fig = px.line(xly_2025, x=xly_2025.index, y="atr_pct", title="XLY ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()

In [39]:
xly = signal_generation(xly, 'XLY')

In [40]:
print(backtest(xly, 'XLY'))

Final Capital: $100729.52 
Total Return: $729.52 
Total Trades: 76
Average Profit per Trade: $9.60


In [41]:
xly_portfolio = f_portfolio(xly, 'XLY')
print(xly_portfolio.stats())
xly_portfolio.plot().show()

Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           170848.194798
Total Return [%]                        70.848195
Benchmark Return [%]                   120.815504
Max Gross Exposure [%]                      100.0
Total Fees Paid                      21558.870642
Max Drawdown [%]                        33.668417
Max Drawdown Duration                       793.0
Total Trades                                   76
Total Closed Trades                            76
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            44.736842
Best Trade [%]                          37.025745
Worst Trade [%]                        -13.070482
Avg Winning Trade [%]                    4.635492
Avg Losing Trade [%]                    -2.172936


/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



## SPY Analysis Results

In [42]:
spy = calculate_true_range(spy, 'SPY')
spy["position_size"] = spy.apply(position_size, axis=1)

# ---- Preview ----
print(spy[["Close_SPY", "ATR_10", "atr_pct", "position_size"]].tail(10))

             Close_SPY    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  611.091675  4.794563  0.007846           0.01
2025-02-20  608.549377  4.806522  0.007898           0.01
2025-02-21  598.140686  5.513399  0.009218           0.01
2025-02-24  595.418884  5.359863  0.009002           0.01
2025-02-25  592.457764  5.718790  0.009653           0.01
2025-02-26  592.756836  6.146507  0.010369           0.01
2025-02-27  583.295288  6.801538  0.011661           0.01
2025-02-28  592.397949  7.353875  0.012414           0.01
2025-03-03  582.019165  8.901222  0.015294           0.01
2025-03-04  575.129883  9.901217  0.017216           0.01


In [43]:
fig = px.line(spy, x=spy.index, y="atr_pct", title="SPY ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()

In [44]:
# Filter only 2025 data
spy_2025 = spy[spy.index.year == 2025]

# Plot
fig = px.line(spy_2025, x=spy_2025.index, y="atr_pct", title="SPY ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()

In [45]:
spy = signal_generation(spy, 'SPY')

In [46]:
print(backtest(spy, 'SPY'))

Final Capital: $100515.03 
Total Return: $515.03 
Total Trades: 56
Average Profit per Trade: $9.20


In [47]:
spy_portfolio = f_portfolio(spy, 'SPY')
print(spy_portfolio.stats())
spy_portfolio.plot().show()

Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           149876.046124
Total Return [%]                        49.876046
Benchmark Return [%]                   153.411688
Max Gross Exposure [%]                      100.0
Total Fees Paid                      14500.381668
Max Drawdown [%]                        19.809446
Max Drawdown Duration                       584.0
Total Trades                                   56
Total Closed Trades                            56
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            55.357143
Best Trade [%]                           7.385099
Worst Trade [%]                         -9.885438
Avg Winning Trade [%]                    3.135409
Avg Losing Trade [%]                    -2.130089


/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



## Optimization (TSLA)

In [48]:
# Define a list of different smoothing periods to test for the Vortex Indicator
periods = [7, 14, 21, 30]
results = {}  # Dictionary to store performance metrics for each period

# Loop through each smoothing period
for n in periods:
    # === Compute Vortex Indicator for the given period ===
    tsla[f'VI+{n}'], tsla[f'VI-{n}'] = calculate_vortex(tsla, 'TSLA', n)

    # === Generate Buy/Sell signals based on crossover logic ===
    # Buy when VI+ crosses above VI-
    tsla[f'Buy_{n}'] = tsla[f'VI+{n}'] > tsla[f'VI-{n}']
    # Sell when VI- crosses above VI+
    tsla[f'Sell_{n}'] = tsla[f'VI-{n}'] > tsla[f'VI+{n}']

    # === Convert boolean signals to actual entry/exit Series ===
    entries = tsla[f'Buy_{n}']
    exits = tsla[f'Sell_{n}']

    # === Run a backtest using vectorbt Portfolio object ===
    portfolio = vbt.Portfolio.from_signals(
        close=tsla['Close_TSLA'],  # TSLA closing prices
        entries=entries,
        exits=exits,
        size=1,  # Assume buying 1 share per trade
        init_cash=10_000  # Initial capital for backtest
    )

    # === Store backtest performance metrics in results dict ===
    stats = portfolio.stats()
    results[n] = stats

# Identify the period with the highest total return
best_period = max(results, key=lambda x: results[x]['Total Return [%]'])
print(f"✅ Best Performing Period: {best_period} days")

# Rebuild portfolio using the best period to visualize it
portfolio = vbt.Portfolio.from_signals(
    close=tsla['Close_TSLA'],
    entries=tsla[f'VI+{best_period}'] > tsla[f'VI-{best_period}'],
    exits=tsla[f'VI-{best_period}'] > tsla[f'VI+{best_period}'],
    size=1,
    init_cash=10_000
)

# Plot the results of the best strategy
portfolio.plot().show()
print(portfolio.stats())


/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requ

✅ Best Performing Period: 7 days


Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                               10000.0
End Value                            10480.194603
Total Return [%]                         4.801946
Benchmark Return [%]                  1215.813231
Max Gross Exposure [%]                   4.554966
Total Fees Paid                               0.0
Max Drawdown [%]                         0.793073
Max Drawdown Duration                       351.0
Total Trades                                  113
Total Closed Trades                           113
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            44.247788
Best Trade [%]                         128.434899
Worst Trade [%]                        -15.721837
Avg Winning Trade [%]                   14.052436
Avg Losing Trade [%]                    -4.125181


/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



## Peer Comparison: Apple Analysis Results 

In [49]:
aapl = yf.download('AAPL', start='2019-01-01', end='2025-03-05')
aapl = multiindex_to_singleindex(aapl)
get_news_sentiment('AAPL', '20250101T0130', '20250301T0130', 1000, 'PNM5EHRALIOT1CKJ')
aapl['VI+'], aapl['VI-'] = calculate_vortex(aapl, 'AAPL')

[*********************100%***********************]  1 of 1 completed


In [50]:
aapl = calculate_true_range(aapl, 'AAPL')
aapl["position_size"] = aapl.apply(position_size, axis=1)

# ---- Preview ----
print(aapl[["Close_AAPL", "ATR_10", "atr_pct", "position_size"]].tail(10))

            Close_AAPL    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  244.869995  4.939392  0.020171           0.01
2025-02-20  245.830002  4.735891  0.019265           0.01
2025-02-21  245.550003  4.746260  0.019329           0.01
2025-02-24  247.100006  4.517000  0.018280           0.01
2025-02-25  247.039993  4.687000  0.018973           0.01
2025-02-26  240.360001  4.719998  0.019637           0.01
2025-02-27  237.300003  4.631998  0.019520           0.01
2025-02-28  241.839996  5.143999  0.021270           0.01
2025-03-03  238.029999  5.479999  0.023022           0.01
2025-03-04  235.929993  5.685001  0.024096           0.01


In [51]:
aapl = signal_generation(aapl, 'AAPL')
# Display the total number of buy and sell signals generated across the dataset
print("Buy signals:", aapl['Buy_Signal'].sum())
print("Sell signals:", aapl['Sell_Signal'].sum())

Buy signals: 985
Sell signals: 552


In [52]:
# Calculate ATR as a percentage of the closing price to normalize volatility
aapl['atr_pct'] = aapl['ATR_10'] / aapl['Close_AAPL']

# Define Vortex Indicator crossover signals:
# - VI_Cross_Up: Identifies when VI+ crosses above VI− (potential bullish signal)
# - VI_Cross_Down: Identifies when VI− crosses above VI+ (potential bearish signal)
aapl['VI_Cross_Up'] = (aapl['VI+'] > aapl['VI-']) & (aapl['VI+'].shift(1) <= aapl['VI-'].shift(1))
aapl['VI_Cross_Down'] = (aapl['VI-'] > aapl['VI+']) & (aapl['VI-'].shift(1) <= aapl['VI+'].shift(1))

# Initialize signal and state columns
aapl['Buy_Signal'] = False          # Flag for buy signal
aapl['Sell_Signal'] = False         # Flag for sell signal
aapl['Position'] = 0                # Position state: 1 = in position, 0 = no position
aapl['Entry_Type'] = None           # Strategy classification: 'aggressive' or 'conservative'

# Initialize control variables for trailing stop and price tracking
in_position = False                 # Boolean flag for current position state
peak_price = 0                      # Highest price observed during an open position

# Iterate through the DataFrame to simulate trading logic based on Vortex signals and volatility
for i in range(1, len(aapl)):
    row = aapl.iloc[i]
    idx = aapl.index[i]

    # Buy condition: Enter a new position if VI_Cross_Up occurs and no current position is held
    if not in_position and row['VI_Cross_Up']:
        aapl.at[idx, 'Buy_Signal'] = True
        aapl.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_AAPL']

        # Classify entry type based on volatility threshold
        if row['atr_pct'] < 0.03:
            aapl.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            aapl.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, evaluate for trailing stop or VI_Cross_Down exit condition
    elif in_position:
        current_price = row['Close_AAPL']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        # Sell condition: Exit if drawdown exceeds 3% or VI_Cross_Down occurs
        if drawdown >= 0.03 or row['VI_Cross_Down']:
            aapl.at[idx, 'Sell_Signal'] = True
            aapl.at[idx, 'Position'] = 0
            in_position = False
        else:
            aapl.at[idx, 'Position'] = 1  # Maintain position

# Output the total count of each type of signal and entry classification
print("Buy signals:", aapl['Buy_Signal'].sum())
print("Sell signals:", aapl['Sell_Signal'].sum())
print("Aggressive entries:", (aapl['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (aapl['Entry_Type'] == 'conservative').sum())

Buy signals: 66
Sell signals: 66
Aggressive entries: 45
Conservative entries: 21


In [53]:
aapl_sentiment_df = json_reader('AAPL')

In [54]:
aapl_sentiment_scores_filtered = aapl_sentiment_df[(aapl_sentiment_df['time_published']).isin(aapl.index)]
aapl_sentiment_scores_filtered = aapl_sentiment_scores_filtered.groupby('time_published')['sentiment_score'].mean().reset_index()

In [55]:
aapl_merged_data = pd.merge(
    aapl['Volume_AAPL'].reset_index().rename(columns={'Volume_AAPL': 'Volume'}),
    aapl_sentiment_scores_filtered,
    left_on='Date',
    right_on='time_published',
    how='inner'
)
# Compute the weighted sentiment by multiplying raw sentiment by trading volume
aapl_merged_data['Weighted_Sentiment'] = aapl_merged_data['Volume'] * aapl_merged_data['sentiment_score']

# Calculate a 5-day rolling average of the weighted sentiment to smooth short-term noise
aapl_merged_data['5_day_avg_sentiment'] = aapl_merged_data['Weighted_Sentiment'].rolling(window=5).mean()

# Define a binary condition for when the average sentiment is positive
aapl_merged_data['Buy_Condition'] = aapl_merged_data['5_day_avg_sentiment'] > 0

# Normalize the rolling sentiment score by average volume to allow comparability across scales
aapl_merged_data['5_day_avg_sentiment_norm'] = (
    aapl_merged_data['5_day_avg_sentiment'] / aapl_merged_data['Volume'].mean()
)

In [56]:
aapl = calculate_true_range(aapl, 'AAPL')
aapl["position_size"] = aapl.apply(position_size, axis=1)

# ---- Preview ----
print(aapl[["Close_AAPL", "ATR_10", "atr_pct", "position_size"]].tail(10))

            Close_AAPL    ATR_10   atr_pct  position_size
Date                                                     
2025-02-19  244.869995  4.939392  0.020171           0.01
2025-02-20  245.830002  4.735891  0.019265           0.01
2025-02-21  245.550003  4.746260  0.019329           0.01
2025-02-24  247.100006  4.517000  0.018280           0.01
2025-02-25  247.039993  4.687000  0.018973           0.01
2025-02-26  240.360001  4.719998  0.019637           0.01
2025-02-27  237.300003  4.631998  0.019520           0.01
2025-02-28  241.839996  5.143999  0.021270           0.01
2025-03-03  238.029999  5.479999  0.023022           0.01
2025-03-04  235.929993  5.685001  0.024096           0.01


In [57]:
aapl_merged_data = pd.merge(
    aapl_merged_data, 
    aapl[['Close_AAPL', 'High_AAPL', 'Low_AAPL', 'Open_AAPL', 'Volume_AAPL',
          'VI+', 'VI-', 'prev_close', 'tr1', 'tr2', 'tr3', 'true_range', 'ATR_10', 'position_size']], 
    on='Date', 
    how='left')
aapl_merged_data.head()

,Date,Volume,time_published,sentiment_score,Weighted_Sentiment,5_day_avg_sentiment,Buy_Condition,5_day_avg_sentiment_norm,Close_AAPL,High_AAPL,...,Volume_AAPL,VI+,VI-,prev_close,tr1,tr2,tr3,true_range,ATR_10,position_size
0,2025-01-15,39832000,2025-01-15,0.223177,8.889575e+06,NaN,False,NaN,237.608749,238.697564,...,39832000,0.595080,1.102317,233.023788,4.525039,5.673775,1.148737,5.673775,5.283191,0.01
1,2025-01-16,71759100,2025-01-16,0.237567,1.704756e+07,NaN,False,NaN,228.009308,237.748600,...,71759100,0.524560,1.139218,237.608749,9.969035,0.139851,9.829185,9.969035,5.895516,0.01
2,2025-01-17,68488300,2025-01-17,0.130304,8.924326e+06,NaN,False,NaN,229.727417,232.034878,...,68488300,0.506950,1.231532,228.009308,3.805813,4.025570,0.219757,4.025570,5.439019,0.01
3,2025-01-21,98070400,2025-01-21,0.169273,1.660064e+07,NaN,False,NaN,222.395477,224.173521,...,98070400,0.514695,1.233423,229.727417,5.034458,5.553896,10.588354,10.588354,6.269107,0.01
4,2025-01-22,64126500,2025-01-22,0.182421,1.169803e+07,1.263203e+07,True,0.231401,223.584167,223.873842,...,64126500,0.570450,1.200538,222.395477,4.325246,1.478365,2.846881,4.325246,6.289084,0.01


In [58]:
# Calculate ATR percentage
aapl_merged_data['atr_pct'] = aapl_merged_data['ATR_10'] / aapl_merged_data['Close_AAPL']

# Vortex crossover logic
aapl_merged_data['VI_Cross_Up'] = (aapl_merged_data['VI+'] > aapl_merged_data['VI-']) & (aapl_merged_data['VI+'].shift(1) <= aapl_merged_data['VI-'].shift(1))
aapl_merged_data['VI_Cross_Down'] = (aapl_merged_data['VI-'] > aapl_merged_data['VI+']) & (aapl_merged_data['VI-'].shift(1) <= aapl_merged_data['VI+'].shift(1))

# Initialize signal & state columns
aapl_merged_data['Buy_Signal'] = False
aapl_merged_data['Sell_Signal'] = False
aapl_merged_data['Position'] = 0
aapl_merged_data['Entry_Type'] = None  # aggressive/conservative

# Trailing stop logic variables
in_position = False
peak_price = 0

for i in range(1, len(aapl_merged_data)):
    row = aapl_merged_data.iloc[i]
    idx = aapl_merged_data.index[i]
    # Buy condition
    if not in_position or row['VI_Cross_Up'] or row['5_day_avg_sentiment_norm']>0:
        aapl_merged_data.at[idx, 'Buy_Signal'] = True
        aapl_merged_data.at[idx, 'Position'] = 1
        in_position = True
        peak_price = row['Close_AAPL']

        # Entry Type: aggressive if ATR < 3%, else conservative
        if row['atr_pct'] < 0.03:
            aapl_merged_data.at[idx, 'Entry_Type'] = 'aggressive'
        else:
            aapl_merged_data.at[idx, 'Entry_Type'] = 'conservative'

    # While in position, check for trailing stop or VI cross down
    elif in_position:
        current_price = row['Close_AAPL']
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price

        if drawdown >= 0.03 or row['VI_Cross_Down']:
            aapl_merged_data.at[idx, 'Sell_Signal'] = True
            aapl_merged_data.at[idx, 'Position'] = 0
            in_position = False
        else:
            aapl_merged_data.at[idx, 'Position'] = 1

# Show result counts
print("Buy signals:", aapl_merged_data['Buy_Signal'].sum())
print("Sell signals:", aapl_merged_data['Sell_Signal'].sum())
print("Aggressive entries:", (aapl_merged_data['Entry_Type'] == 'aggressive').sum())
print("Conservative entries:", (aapl_merged_data['Entry_Type'] == 'conservative').sum())

Buy signals: 28
Sell signals: 1
Aggressive entries: 23
Conservative entries: 5


In [59]:
# Ensure 'Date' is datetime and set as index if needed
aapl_merged_data['Date'] = pd.to_datetime(aapl_merged_data['Date'])

fig = go.Figure()

# Plot 5-day Avg Sentiment
fig.add_trace(go.Scatter(
    x=aapl_merged_data['Date'],
    y=aapl_merged_data['5_day_avg_sentiment_norm'],
    mode='lines+markers',
    name='5-Day Avg Sentiment',
    line=dict(color='blue')
))

# Plot ATR %
fig.add_trace(go.Scatter(
    x=aapl_merged_data['Date'],
    y=aapl_merged_data['atr_pct'],
    mode='lines+markers',
    name='ATR %',
    yaxis='y2',
    line=dict(color='orange')
))

# Optional: Highlight Buy Signal Dates (even though there are none now)
fig.add_trace(go.Scatter(
    x=aapl_merged_data.loc[aapl_merged_data['Buy_Signal'], 'Date'],
    y=aapl_merged_data.loc[aapl_merged_data['Buy_Signal'], '5_day_avg_sentiment_norm'],
    mode='markers',
    marker=dict(color='green', size=10, symbol='star'),
    name='Buy Signal'
))

# Add dual axis layout
fig.update_layout(
    title="5-Day Sentiment vs ATR % (with Buy Signals)",
    xaxis_title='Date',
    yaxis=dict(title='5-Day Avg Sentiment'),
    yaxis2=dict(title='ATR %', overlaying='y', side='right'),
    legend=dict(x=0.01, y=0.99),
    height=500
)

fig.show()

In [60]:
print(backtest(aapl_merged_data, 'AAPL')) #w/ sentiment data

Final Capital: $100057.01 
Total Return: $57.01 
Total Trades: 1
Average Profit per Trade: $-24.62


In [61]:
print(backtest(aapl, 'AAPL')) #w/o sentiment data

Final Capital: $101198.29 
Total Return: $1198.29 
Total Trades: 66
Average Profit per Trade: $18.16


In [62]:
# without centiment data
aapl_portfolio = f_portfolio(aapl, 'AAPL')

print(aapl_portfolio.stats())
aapl_portfolio.plot().show()

Start                         2019-01-02 00:00:00
End                           2025-03-04 00:00:00
Period                                       1551
Start Value                              100000.0
End Value                           255472.954051
Total Return [%]                       155.472954
Benchmark Return [%]                   526.354355
Max Gross Exposure [%]                      100.0
Total Fees Paid                        23330.2112
Max Drawdown [%]                        14.949616
Max Drawdown Duration                       238.0
Total Trades                                   66
Total Closed Trades                            66
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            48.484848
Best Trade [%]                          18.319731
Worst Trade [%]                         -5.776766
Avg Winning Trade [%]                     5.44886
Avg Losing Trade [%]                    -2.071776


/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/Users/dinarazhorabek/Library/Python/3.9/lib/python/site-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Based on the results from applying the trading strategy to the Apple (AAPL) ticker, we can reasonably conclude that the strategy does work on peers like AAPL. The strategy delivered a total return of approximately 282% over the backtest period (2019–2025), compared to a benchmark return of about 526%, which indicates it captured a significant portion of the upward trend while actively managing trades. Although it underperformed the benchmark in absolute terms, this is typical of signal-driven strategies that trade in and out of the market. The profit factor of 2.11, expectancy of 4204, and a win rate of 45.5% suggest the strategy was profitable overall. Additionally, the drawdown was moderate (20.87%), reflecting a reasonable risk exposure relative to the potential reward.

The cumulative returns graph further supports this interpretation. The strategy closely follows the broader market trend, generating consistent gains and outperforming during certain periods. The trade PnL distribution shows a good number of winning trades with healthy profitability, and although there were losses, the downside was generally contained. Therefore, this peer comparison confirms that the strategy generalizes reasonably well beyond TSLA, making it a potentially viable approach for other high-liquidity technology stocks like AAPL.